In [1]:
from pyspark.sql.functions import substring_index, col
from pyspark.sql import SparkSession, Row
import re
import pandas as pd
from pyspark.sql import functions as F, Window
import unidecode
from pyspark.sql.functions import isnan, when, count, col, isnull

#############################################
#            read data                      # 
#############################################

file = "/data/SMARTDATA/books/BCT_prestiti/items.csv"
DFitems = spark.read.csv(file,header=True)
# DFitems.show(5, False)

file = "/data/SMARTDATA/books/BCT_prestiti/item_media_types.csv"
DFtypes = spark.read.csv(file,header=True)
# DFtypes.show()

file = "/data/SMARTDATA/books/BCT_prestiti/manifestations.csv"
DFmanifestations = spark.read.csv(file,header=True)

# DFmanifestations = DFmanifestations.filter(DFmanifestations.title.contains("Il mio credo"))
# DFmanifestations.show(5, False)

file = "/data/SMARTDATA/books/BCT_prestiti/loans.csv"
DFloans = spark.read.csv(file,header=True)
# DFloans.show()

In [2]:
#############################################
#   this part remove from manifestation     #
#              items and loans              #
#    things that are not manuscript and     #
#             monografie                    #
#############################################
DFtypesitems = DFtypes.join(DFitems, DFitems.item_media_id == DFtypes.item_media_id).drop(DFitems.item_media_id)

#Remove in DFitems books of types we are not intersted in
DFitems_filtered_obj = DFitems.filter((DFitems.item_media_id =="B") | (DFitems.item_media_id == "F"))
# tale manifestation_if of books that are kept
manifestation_id_to_keep =  DFitems_filtered_obj.select('manifestation_id').distinct()
# DFitems_filtered.show()
#manifestation_id_to_keep.show()

#Remove in DFloans loans of books of types we are not intersted in
DFloans_filtered_obj = DFloans.join(DFitems_filtered_obj, DFloans["item_id"] == DFitems_filtered_obj["item_id"], "leftsemi")
# DFloans_filtered.show()
# DFloans.filter(DFloans.item_id == 288788).show() #item_id di un Q
# DFloans_filtered.filter(DFloans_filtered.item_id == 288788).show() #item_id di un Q

#Remove in DFmanifestations books of types we are not intersted in
DFmanifestations_filtered_obj = DFmanifestations.join(manifestation_id_to_keep, \
                                                  DFmanifestations["manifestation_id"] == manifestation_id_to_keep["manifestation_id"], "leftsemi")
# DFmanifestations_filtered.show()
print("Togli oggetti inutili: Prima "+str(DFmanifestations.count())+"  - Dopo "+str(DFmanifestations_filtered_obj.count())) 
#Prima 290125  - Dopo 257054

#############################################
#  this part remove books which are not     #
#                 in italian                #
#       and Topolino and Dylan Dog          #
#############################################
#I checked that the possibilities are ITA or ita for the italian language
#remove in DFmanifestation
DFmanifestations_filtered_ita = DFmanifestations_filtered_obj.filter( (DFmanifestations_filtered_obj.edition_language == "ITA") |\
                                                                 (DFmanifestations_filtered_obj.edition_language == "ita") )
# DFmanifestations_filtered.show()
print("Togli libri in lingua straniera: Prima "+str(DFmanifestations.count())+"  - Dopo "+str(DFmanifestations_filtered_ita.count())) 
DFmanifestations_filtered_comics = DFmanifestations_filtered_ita.filter(~((DFmanifestations_filtered_ita.title.contains("Topolino")) |
                                                               (DFmanifestations_filtered_ita.title.contains("Zagor")) |
                                                               (DFmanifestations_filtered_ita.title.contains("Dylan Dog")) | 
                                                               (DFmanifestations_filtered_ita.title.contains("Tex")) | 
                                                               (DFmanifestations_filtered_ita.title.contains("Paperino")) | 
                                                               (DFmanifestations_filtered_ita.title.contains("Disney   Big"))))
print("Togli Fumetti: Prima "+str(DFmanifestations.count())+"  - Dopo "+str(DFmanifestations_filtered_comics.count())) 
DFitems_filtered_comics = DFitems_filtered_obj.join(DFmanifestations_filtered_comics, DFmanifestations_filtered_comics["manifestation_id"] == DFitems_filtered_obj["manifestation_id"], \
                                                    "leftsemi")
DFloans_filtered_comics = DFloans_filtered_obj.join(DFitems_filtered_comics, DFloans_filtered_obj["item_id"] == DFitems_filtered_comics["item_id"], \
                                                    "leftsemi")

Togli oggetti inutili: Prima 290125  - Dopo 257054
Togli libri in lingua straniera: Prima 290125  - Dopo 228059
Togli Fumetti: Prima 290125  - Dopo 227749


In [3]:
#PULIZIA AUTORI E TITOLI
def mapping(line):
    author_cleaned = line.author #AUTORI CON UN SOLO NOME (DI SOLITO PSEUDONIMI)
    string_value = str(line.ISBNISSN).split(" ")[0]
    ISBNISSN = ''.join(ch for ch in string_value if ch.isalnum())
    if line.author != None:
        if len(line.author.split(" - ")) != 1: #AUTORI MULTIPLI
            author_cleaned = ""
            count = 0
            for author in line.author.split(" - "):
                count += 1
                if len(author.split(", ")) == 2:
                    if count != len(line.author.split(" - ")):
                        author_cleaned += author.split(", ")[1] + " " + author.split(", ")[0] + " - "
                    else:
                        author_cleaned += author.split(", ")[1] + " " + author.split(", ")[0]
                else:
                    if count != len(line.author.split(" - ")):
                        author_cleaned += author.split(", ")[0] + " - "
                    else:
                        author_cleaned += author.split(", ")[0]
        else:
            if len(line.author.split(",")) == 2: #AUTORI CON NOME E COGNOME SEMPLICI
                author_cleaned = line.author.split(", ")[1] + " " + line.author.split(", ")[0]
            
    else: #L'autore è contenuto nel titolo e non nel campo autore
        author_cleaned = ""
        title_zone_temp = line.title.split(" ; ")[0]
        if len(title_zone_temp.split(" / ")) == 2:
            if len(title_zone_temp.split(" / ")[1].split(" e ")) > 1: #AUTORI SEPARATI DA CONGIUNZIONE E
                title_zone = title_zone_temp.split(" / ")[1].split(" e ")
                count = 0
                for part in title_zone:
                    count += 1
                    if count > 1 and count <= len(title_zone):
                        author_cleaned += " - "
                    words = part.split(" ")
                    count_word = 0 
                    for word in words:
                        count_word += 1
                        string_word = str(word)
                        if len(string_word) > 0:
                            if string_word[0].strip(" [").isupper() == True:
                                if count_word == len(words):
                                    author_cleaned += string_word
                                else:
                                    author_cleaned += string_word + " "
                if author_cleaned.startswith(" -"): #NEL CASO CI FOSSE UNA "e" CHE NON SEPARA DUE AUTORI
                    author_cleaned = author_cleaned.replace(" -", "")
            else:
                title_zone = title_zone_temp.split(" / ")[1].replace("[", "").split(" ")
                if len(title_zone) > 0:
                    count = 0
                    for word in title_zone:
                        string_word = str(word)
                        count += 1
                        if len(string_word) > 0:
                            if string_word[0].isupper() == True:
                                if count == len(title_zone):
                                    author_cleaned += string_word
                                else:
                                    author_cleaned += string_word + " "
                        if len(author_cleaned.split(", ")) > 1: #Autori multipli con virgola
                            templist = author_cleaned.split(", ")
                            author_cleaned = ""
                            for author in templist:
                                string_word = str(author)
                                count += 1
                                if len(string_word) > 0:
                                    if string_word[0].isupper() == True:
                                        if count == len(title_zone):
                                            author_cleaned += string_word
                                        else:
                                            author_cleaned += string_word + " - "
    if author_cleaned.startswith(" "):
        author_cleaned = author_cleaned.replace(" ", "", 1)
    author_cleaned = author_cleaned.replace("*", "")
    author_cleaned = author_cleaned.replace('"', "")
    author_cleaned = author_cleaned.replace(":", "")
    author_cleaned = author_cleaned.replace("!", "")
    author_cleaned = author_cleaned.strip(' []*:') #PER PULIRE LE EVENTUALI PARENTESI QUADRE E SPAZI  
    author_cleaned = re.sub('<[^>]+>', '', author_cleaned)
    author_cleaned = author_cleaned.replace("<", "")
    author_cleaned = re.sub(' +',' ', author_cleaned)
    author_cleaned = author_cleaned.replace("_", " ")
    author_cleaned = author_cleaned.replace(">", "").strip(" ")
    
    #NUOVE RIPULITURE SULL'AUTORE
    author_cleaned = author_cleaned.replace("C. Collodi", "Carlo Collodi").strip(" ")
    if author_cleaned == "Collodi":
        author_cleaned = author_cleaned.replace("Collodi", "Carlo Collodi").strip(" ")
    auhtor_cleaned = author_cleaned.replace("; ", " - ")
    author_cleaned = author_cleaned.strip(".")
    author_cleaned = author_cleaned.strip(" -")
    author_cleaned = author_cleaned.strip(")")
    author_cleaned = author_cleaned.replace("...et", "")
    author_cleaned = author_cleaned.replace("[et", "")
    author_cleaned = author_cleaned.strip(".")
    author_cleaned = author_cleaned.replace(",", " - ").strip(" ")
    author_cleaned = author_cleaned.replace(" -  ", " - ").strip(" ")
    author_cleaned = author_cleaned.replace(" -  - ", " - ").strip(" ")
    if author_cleaned == "Teresa Czerniewicz-Umer - Malgorzata OmilanowskaJerzy S. Majewski":
        author_cleaned = "Teresa Czerniewicz-Umer - Malgorzata Omilanowska - Jerzy S. Majewski"
    if author_cleaned == "Vittorio Marsiglia Carlo Missaglia":
        author_cleanded = "Vittorio Marsiglia - Carlo Missaglia"
    if author_cleaned == "Bert Hellinger - Gabriella Gabriele]Hövel":
        author_cleaned = "Bert Hellinger - Gabriella Gabriele Hövel"
    
    #PER ELIMINARE I REFUSI PIù FREQUENTI
    title_cleaned = line.title.split(" / ")[0]
    title_cleaned = title_cleaned.replace("\x89", "").strip(" ")
    title_cleaned = title_cleaned.replace("\x88", "").strip(" ")
    title_cleaned = title_cleaned.replace("\\1", "").strip(" ")
    title_cleaned = title_cleaned.replace("\\2", "").strip(" ")
    title_cleaned = title_cleaned.replace('"', "")
    title_cleaned = title_cleaned.replace('[', "")
    title_cleaned = title_cleaned.replace("\\", "").strip(" ")
    title_cleaned = title_cleaned.replace(']', "")
    title_cleaned = title_cleaned.replace('*', "")
    title_cleaned = title_cleaned.replace('|', "")
    title_cleaned = title_cleaned.replace("!:", "").strip(" ")
    title_cleaned = title_cleaned.replace(" : romanzo", "")
    return Row(line.manifestation_id, line.edition_language, line.edition_date,
               title_cleaned, author_cleaned, line.publisher, ISBNISSN)

def specialcasesmapping(line): #Casi strani disseminati nel Database che riparo manualmente
    new_author = line.author
    new_title = line.title
    new_publisher = line.publisher
    if line.manifestation_id == "440293":
        new_author = "Chiara Lossano"
    if line.manifestation_id == "190617":
        new_author = "Italo Calvino"
    if line.manifestation_id == "430234":
        new_publisher = None
        new_author = "Philip Roth"
        new_title = "«Ho sempre voluto che ammiraste il mio digiuno» ovvero, guardando Kafka"
    if line.manifestation_id == "427622":
        new_author = "Shí Héng Chán - Shì Héng Díng"
    return Row(line.manifestation_id, line.edition_language, line.edition_date,
                   new_title, new_author, line.publisher, line.ISBNISSN)

#clean title and authors
DFmanifestations_filtered_comics = DFmanifestations_filtered_comics.rdd.map(lambda x:mapping(x, ))\
                                .toDF(["manifestation_id","edition_language","edition_date","title","author","publisher", "ISBNISSN"])
DFmanifestations_filtered = DFmanifestations_filtered_comics.rdd.map(lambda x:specialcasesmapping(x, ))\
                                .toDF(["manifestation_id","edition_language","edition_date","title","author","publisher", "ISBNISSN"])
# DFmanifestations_filtered.show(20, False)

#remove row which does not have the author
DFmanifestations_filtered = DFmanifestations_filtered.filter(DFmanifestations_filtered.author != "")

In [4]:
# DFmanifestations_filtered.show(2, False)

+----------------+----------------+------------+----------------------------------------------------------------+----------------+------------------+-------------+
|manifestation_id|edition_language|edition_date|title                                                           |author          |publisher         |ISBNISSN     |
+----------------+----------------+------------+----------------------------------------------------------------+----------------+------------------+-------------+
|202094          |ita             |2003        |Okey dokey, sono un punk                                        |Domenica Luciani|Feltrinelli       |8807920638   |
|910309          |ita             |2017        |Che cosa ho in testa : immagini di un mondo in cui valga la pena|AA. VV          |Baldini & Castoldi|9788893880442|
+----------------+----------------+------------+----------------------------------------------------------------+----------------+------------------+-------------+
only showing top

In [4]:
#############################################
#     this part gives to the same title     # 
#       the same manifestation_id_new       #
#                SOLO TITOLO                #
#############################################
# DFmanifestations_filtered.filter(DFmanifestations_filtered.title == "Il mio credo").show()
DFmanifestations_filtered_title = DFmanifestations_filtered.select(
    "*",
    F.min('manifestation_id').over(Window.partitionBy('title')).alias('manifestation_id_new')
)
# DFmanifestations_filtered_title.filter(DFmanifestations_filtered.title == "Il mio credo").show()

DFmanifestations_filtered_title = DFmanifestations_filtered_title.select(
    "*",
    F.min('ISBNISSN').over(Window.partitionBy('title')).alias('ISBNISSN_new')
)
# DFmanifestations_filtered.show()
# DFmanifestations_filtered_title.filter(DFmanifestations_filtered.title == "Il mio credo").show()

#############################################
#    this part to remove books with less    # 
#              than 50 loans                #
#############################################
#add manifestation_id_new in the item table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
 #remove the rows in DFitems but not in DFmanifestation
DFitems_tmp_title = DFitems.join(DFmanifestations_filtered_title, ['manifestation_id'])
#remove useless (in this part) columns
col_to_drop_title = ("manifestation_id", "title", "author", "publisher", "home_library_id", "inventory_date", "edition_language", "item_media_id", "edition_date")
DFitems_dropped_title = DFitems_tmp_title.drop(*col_to_drop_title)
# DFitems_dropped_title.show()

#add manifestation_id_new in the loan table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
DFloans_tmp_title = DFloans.join(DFitems_dropped_title, ['item_id'])
# DFloans_tmp.show()

#remove useless (in this part) columns
col_to_drop_title = ("patron_id_md5", "loan_date_begin", "loan_date_end", "due_date", "from_library", "to_library", "end_library")
DFloans_dropped_title = DFloans_tmp_title.drop(*col_to_drop_title)
# DFloans_dropped_title.show()

#groupby sui loans e conteggio prestiti di ogni libro N.B la groupby è fatta sul manifestation_id perchè voglio sapere di un libro e NON di UNA COPIA DI QUEL LIBRO!!!
DFloans_with_item_id_counter_title = DFloans_dropped_title.groupBy("manifestation_id_new").agg(F.count('manifestation_id_new').alias('counter'))#count().alias("counter").show()
# DFloans_with_item_id_counter.show()
# DFloans_with_item_id_counter.filter(DFloans_with_item_id_counter.manifestation_id_new == 118878).show() #sette prestiti

#select only manifestation_id_new with at least 50 loans
DFloans_with_item_id_counter_morethan50_title = DFloans_with_item_id_counter_title.filter(DFloans_with_item_id_counter_title.counter >= 50).select("manifestation_id_new").distinct()
# DFloans_with_item_id_counter_morethan50.show()

#join with DFmanifestation to keep in the manifestation table books with at least 50 loans
DFmanifestations_filtered_aggregated_title = DFmanifestations_filtered_title.join(DFloans_with_item_id_counter_morethan50_title, 
                                                                      DFmanifestations_filtered_title["manifestation_id_new"] == DFloans_with_item_id_counter_morethan50_title["manifestation_id_new"],
                                                                      "leftsemi")

print("prefiltrati: "+str(DFmanifestations_filtered.count())+" - filtrati: "+\
      str(DFmanifestations_filtered_aggregated_title.select(DFmanifestations_filtered_aggregated_title.manifestation_id_new).distinct().count())) #prefiltrati: 218419 - filtrati: 17947



# DFmanifestations_filtered_aggregated.select(DFmanifestations_filtered_aggregated.author).toPandas().to_csv('author.csv')

#############################################
#     this part gives to the same title     # 
#       the same manifestation_id_new       #
#           SOLO TITOLO E AUTORE            #
#############################################
# DFmanifestations_filtered.filter(DFmanifestations_filtered.title == "Il mio credo").show()
column_list = ['title', 'author']
from pyspark.sql.functions import col
DFmanifestations_filtered_title_author = DFmanifestations_filtered.select(
    "*",
    F.min('manifestation_id').over(Window.partitionBy([col(x) for x in column_list])).alias('manifestation_id_new')
)
DFmanifestations_filtered_title_author = DFmanifestations_filtered_title_author.select(
    "*",
    F.min('ISBNISSN').over(Window.partitionBy([col(x) for x in column_list])).alias('ISBNISSN_new')
)
# DFmanifestations_filtered.show()
# DFmanifestations_filtered_title_author.filter(DFmanifestations_filtered.title == "Il mio credo").show()

#############################################
#    this part is to find manifestation     # 
#         with less than 50 loans           #
#############################################
#add manifestation_id_new in the item table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
 #remove the rows in DFitems but not in DFmanifestation
DFitems_tmp_title_author = DFitems.join(DFmanifestations_filtered_title_author, ['manifestation_id'])
#remove useless (in this part) columns
col_to_drop_title_author = ("manifestation_id", "title", "author", "publisher", "home_library_id", "inventory_date", "edition_language", "item_media_id", "edition_date")
DFitems_dropped_title_author = DFitems_tmp_title_author.drop(*col_to_drop_title_author)
# DFitems_dropped_title_author.show()

#add manifestation_id_new in the loan table
# DFitems.join(DFmanifestations_filtered, ['manifestation_id']).show()
DFloans_tmp_title_author = DFloans.join(DFitems_dropped_title_author, ['item_id'])
# DFloans_tmp.show()

#remove useless (in this part) columns
col_to_drop_title_author = ("patron_id_md5", "loan_date_begin", "loan_date_end", "due_date", "from_library", "to_library", "end_library")
DFloans_dropped_title_author = DFloans_tmp_title_author.drop(*col_to_drop_title_author)
# DFloans_dropped_title_author.show()

#groupby sui loans e conteggio prestiti di ogni libro N.B la groupby è fatta sul manifestation_id perchè voglio sapere di un libro e NON di UNA COPIA DI QUEL LIBRO!!!
DFloans_with_item_id_counter_title_author = DFloans_dropped_title_author.groupBy("manifestation_id_new").agg(F.count('manifestation_id_new').alias('counter'))#count().alias("counter").show()
# DFloans_with_item_id_counter.show()
# DFloans_with_item_id_counter.filter(DFloans_with_item_id_counter.manifestation_id_new == 118878).show() #sette prestiti

#select only manifestation_id_new with at least 50 loans
DFloans_with_item_id_counter_morethan50_title_author = DFloans_with_item_id_counter_title_author.filter(DFloans_with_item_id_counter_title_author.counter >= 50).select("manifestation_id_new").\
                                                       distinct()
# DFloans_with_item_id_counter_morethan50.show()

#join with DFmanifestation to keep in the manifestation table books with at least 50 loans
DFmanifestations_filtered_aggregated_title_author = DFmanifestations_filtered_title_author.join(DFloans_with_item_id_counter_morethan50_title_author, 
                                                                      DFmanifestations_filtered_title_author["manifestation_id_new"] == DFloans_with_item_id_counter_morethan50_title_author["manifestation_id_new"],
                                                                      "leftsemi")

print("Manifestation prefiltrati: "+str(DFmanifestations_filtered.count())+" - filtrati: "+\
      str(DFmanifestations_filtered_aggregated_title_author.select(DFmanifestations_filtered_aggregated_title_author.manifestation_id_new).distinct().count())) #prefiltrati: 218419 - filtrati: 18164

# DFmanifestations_filtered_aggregated.select(DFmanifestations_filtered_aggregated.author).toPandas().to_csv('author.csv')

prefiltrati: 218419 - filtrati: 17947
Manifestation prefiltrati: 218419 - filtrati: 18164


In [5]:
#to verify how many null and nan are present in the ISBNISSN_new, ISBNISSN field anymore
# DFmanifestations_filtered_aggregated_title_author.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["ISBNISSN"]]).show()
# DFmanifestations_filtered_aggregated_title_author.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["ISBNISSN_new"]]).show()
# DFmanifestations.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["ISBNISSN"]]).show()

DFmanifestations_final = DFmanifestations_filtered_aggregated_title_author
DFitems_final = DFitems_filtered_comics.join(DFmanifestations_final, DFmanifestations_final["manifestation_id"] == DFitems_filtered_comics["manifestation_id"], \
                                                    "leftsemi")
DFloans_final = DFloans_filtered_comics.join(DFitems_final, DFloans_filtered_comics["item_id"] == DFitems_final["item_id"], \
                                                    "leftsemi")

print("Item prefiltrati: "+str(DFitems.count())+" - filtrati: "+\
      str(DFitems_final.count())) #prefiltrati: 998403 - filtrati: 217405
print("Loan prefiltrati: "+str(DFloans.count())+" - filtrati: "+\
      str(DFloans_final.count())) #prefiltrati: 5484078 - filtrati: 2172971

# DFmanifestations_final.show(2, False)
# DFitems_final.show(2, False)
# DFloans_final.show(2, False)

Item prefiltrati: 998403 - filtrati: 217405
Loan prefiltrati: 5484078 - filtrati: 2172971


In [6]:
#substitute manifestation_id with manifestation_id_new
DFitems_definitive = DFitems_final.join(DFmanifestations_final,DFmanifestations_final["manifestation_id"] == DFitems_final["manifestation_id"], "inner")\
                                  .select(DFmanifestations_final.manifestation_id_new, \
                                          DFitems_final.item_id, \
                                          DFitems_final.item_media_id,\
                                          DFitems_final.home_library_id,\
                                          DFitems_final.inventory_date)
# DFitems_definitive.show(2, False)
#substitute manifestation_id with manifestation_id_new
DFloans_definitive = DFloans_final.join(DFitems_definitive, DFitems_definitive["item_id"] == DFloans_final["item_id"], "inner")\
                                  .select(DFitems_definitive.manifestation_id_new, \
                                          DFloans_final.patron_id_md5, \
                                          DFloans_final.loan_date_begin,\
                                          DFloans_final.loan_date_end,\
                                          DFloans_final.due_date,\
                                          DFloans_final.from_library,\
                                          DFloans_final.to_library,\
                                          DFloans_final.end_library,)
# DFloans_definitive.show(2, False)

#DFmanifestations_definitive contains only manifestation_id and ISBNISSN
DFmanifestations_definitive = DFmanifestations_final.groupby("manifestation_id_new").agg(F.min("manifestation_id").alias("manifestation_id"),\
                                                                                         F.min("ISBNISSN").alias("ISBNISSN")).drop("manifestation_id_new").drop("ISBNISSN_new")#\
# DFmanifestations_final.show(2, False)
# DFmanifestations_definitive.show(2, False)
#per avere tutti gli attributi in DFmanifestations_definitive e togliere manifestation_id e ISBNISSN e tenere solo manifestation_id_new e ISBNISSN_new 
DFmanifestations_definitive = DFmanifestations_definitive.join(DFmanifestations_final, DFmanifestations_final["manifestation_id"] == DFmanifestations_definitive["manifestation_id"])\
                                                         .drop(DFmanifestations_final["manifestation_id"]).drop(DFmanifestations_definitive["manifestation_id"])\
                                                         .drop(DFmanifestations_final["ISBNISSN"]).drop(DFmanifestations_definitive["ISBNISSN"])
# DFmanifestations_definitive.show(2, False)
print("i libri sono: "+str(DFmanifestations_definitive.count())) #--> i libri sono: 18164
# DFmanifestations_definitive.count() deve essere uguale a DFmanifestations_final.select("manifestation_id_new").distinct().count()

i libri sono: 18164


In [7]:
# DFmanifestations_final.filter(DFmanifestations_final.title == "Pinocchio").show(3, False)
# DFmanifestations_definitive.filter(DFmanifestations_definitive.title == "Pinocchio").show(3, False)

+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+------------+
|manifestation_id|edition_language|edition_date|title    |author       |publisher  |ISBNISSN     |manifestation_id_new|ISBNISSN_new|
+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+------------+
|658351          |ita             |2016        |Pinocchio|Carlo Collodi|Gribaudo   |9788858014554|107930              |8807820714  |
|193757          |ita             |1972        |Pinocchio|Carlo Collodi|Feltrinelli|None         |107930              |8807820714  |
|314571          |ita             |2006        |Pinocchio|Carlo Collodi|Giunti kids|9788809046085|107930              |8807820714  |
+----------------+----------------+------------+---------+-------------+-----------+-------------+--------------------+------------+
only showing top 3 rows

+----------------+------------+---------+---

In [9]:
DFmanifestations_definitive.toPandas().to_csv("manifestationCleaned.csv")
DFloans_definitive.toPandas().to_csv("loansCleaned.csv")